In [1]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

In [2]:
def grab_symbols():
    # Grab S&P Symbols from Wikipedia or local HTML File
    # wiki_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    # wiki_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#S&P_500_component_stocks'
    tickers = pd.read_html('./tickers.html')[0]
    tickers = tickers.Symbol.to_list()
    tickers = [i.replace('.','-') for i in tickers]
    return tickers 

print('Grab symbols function defined...')

Grab symbols function defined...


In [3]:
def yahoo_prices(tickers):
    data = yf.download(tickers, start='2011-01-01')
    data = data.loc[(slice(None)),(slice(None),slice(None))].copy()
    data = data.stack()
    data = data.reset_index()
    data.rename(columns={'level_1': 'Symbol'}, inplace=True)
    data.set_index('Date', inplace=True)
    return data

print('Yahoo_prices function defined...')    

Yahoo_prices function defined...


In [5]:
# Generate our indicators and buy#sell signals for each of our assets
# df: contains one company (symbol) of data
def RSIcalc(df):
    # Sanity check for our indicators to work
    if len(df) < 250:
        return None
    
    df['MA200'] = df['Adj Close'].rolling(window=200).mean()
    df['Price_change'] = df['Adj Close'].pct_change()

    df['Upmove']   = df['Price_change'].apply(lambda x: x if x > 0 else 0)
    df['Downmove'] = df['Price_change'].apply(lambda x: abs(x) if x < 0 else 0)
    df['Avg_up']   = df['Upmove'].ewm(span=19).mean()
    df['Avg_down']   = df['Downmove'].ewm(span=19).mean()
    df = df.dropna().copy()
    df['RS'] = df['Avg_up'] / df['Avg_down']
    df['RSI'] = df['RS'].apply(lambda x: 100 - (100 / (x + 1)))

    df.loc[(df['Adj Close'] > df['MA200']) & (df['RSI'] < 30), 'Buy' ] = 'Yes'
    df.loc[(df['Adj Close'] <= df['MA200']) | (df['RSI'] >= 30), 'Buy' ] = 'No'

    return df
    
print('RSIcalc function defined....')

RSIcalc function defined....


In [6]:
# Extract Buy & Sell dates from our Asset Dataframe
# df: contains one company (symbol) of data
def get_signals(df):
    Buying_dates = []
    Selling_dates = []
    for i in range(len(df) - 11):
        if 'Yes' in df['Buy'].iloc[i]:
            Buying_dates.append(df.iloc[i+1].name)  # name is the index that contains the date
            for j in range(1,11):
                if df['RSI'].iloc[i + j] > 40:
                    Selling_dates.append(df.iloc[i + j + 1].name)
                    break # Break before we reach the tenth day
                elif j == 10:
                    Selling_dates.append(df.iloc[i + j + 1].name)
    return Buying_dates, Selling_dates  

print('get_signals function defined....')  


get_signals function defined....


In [7]:
tickers = grab_symbols()
all_assets = yahoo_prices(tickers)

# Very long to run, will create a large file on system, BE CAREFUL
#all_assets.to_csv('SP_500_daily_prices.csv')

# Utility function to load prices from our saved historical file
#sp500 = pd.read_csv('SP_500_daily_prices.csv', index_col='Date')
#sp500.loc[sp500.Symbol == 'TSLA']

print('All S&P Prices loaded/downloaded')

[*********************100%***********************]  502 of 502 completed
All S&P Prices loaded/downloaded


In [8]:
# Generate a list of trades for one specific company (symbol) based on our buy/sell signals
def log_trades(asset_frame):
    symbol = asset_frame.iloc[0].Symbol
    buy, sell = get_signals(asset_frame)
    buy_dates = asset_frame.loc[buy].index.tolist()
    buy_prices = asset_frame.loc[buy].Open.values.tolist()
    sell_dates = asset_frame.loc[sell].index.tolist()
    sell_prices = asset_frame.loc[sell].Open.values.tolist()

    trades = pd.DataFrame(buy_dates, columns=['Buy_dates'])
    trades['Buy_price'] = buy_prices
    trades['Sell_date'] = sell_dates
    trades['Sell_price'] = sell_prices
    trades['Overlaping_trades'] = [same_sell_date for same_sell_date in trades['Sell_date'] == trades['Sell_date'].shift(1)]
    trades['Profit_amount'] = trades['Sell_price'] - trades['Buy_price']
    trades['Profit'] = [amount for amount in trades['Profit_amount'] > 0]
    trades['Symbol'] = symbol
    
    # Remove overlapping trades
    trades = trades.loc[trades.Overlaping_trades == False]
    
    # Move Symbol to first column position for easy reading :)
    first_column = trades.pop('Symbol')
    trades.insert(0, 'Symbol', first_column)
    return trades

print('log_trades function defined....')

log_trades function defined....


In [9]:
tickers.index('TSLA')

445

In [10]:
# Test our functions for only one stock before applying our trade log to all our 503 assets
i = 445
data = all_assets.loc[all_assets.Symbol == tickers[i]].copy()
frame = RSIcalc(data)
if frame is not None:
    trades = log_trades(frame)

trades

,Symbol,Buy_dates,Buy_price,Sell_date,Sell_price,Overlaping_trades,Profit_amount,Profit
0,TSLA,2011-12-19 00:00:00-05:00,1.872667,2011-12-28 00:00:00-05:00,1.932667,False,0.060000,True
3,TSLA,2013-11-08 00:00:00-05:00,9.098667,2013-11-22 00:00:00-05:00,8.105333,False,-0.993334,False
4,TSLA,2013-11-11 00:00:00-05:00,9.400000,2013-11-25 00:00:00-05:00,8.300000,False,-1.099999,False
5,TSLA,2013-11-19 00:00:00-05:00,7.962000,2013-11-29 00:00:00-05:00,8.651333,False,0.689333,True
11,TSLA,2014-05-09 00:00:00-04:00,11.990667,2014-05-14 00:00:00-04:00,12.596667,False,0.606000,True
12,TSLA,2014-10-02 00:00:00-04:00,16.680000,2014-10-03 00:00:00-04:00,16.870667,False,0.190666,True
13,TSLA,2014-10-14 00:00:00-04:00,15.216667,2014-10-22 00:00:00-04:00,15.546000,False,0.329332,True
15,TSLA,2017-07-06 00:00:00-04:00,21.150667,2017-07-12 00:00:00-04:00,22.026667,False,0.875999,True
18,TSLA,2017-10-26 00:00:00-04:00,21.851999,2017-11-01 00:00:00-04:00,22.150000,False,0.298000,True
22,TSLA,2018-12-21 00:00:00-05:00,21.160000,2018-12-27 00:00:00-05:00,21.322666,False,0.162666,True


In [11]:
# Run our analysis on all our S&P500 companies
all_trades = None

len_tickers = len(tickers)
for i in range(len_tickers):
    data = all_assets.loc[all_assets.Symbol == tickers[i]].copy()
    frame = RSIcalc(data)
    if frame is not None:
        trades = log_trades(frame)
    winning_trades = trades.Profit.sum()
    print(f'[{i}/{len_tickers}][{trades.iloc[0].Symbol}] Winning trades:{winning_trades}/{len(trades)} for total profit of ${trades.Profit_amount.sum():0.2f}')

    if all_trades is None:
        all_trades = trades.copy()
    else:
        all_trades = pd.concat([all_trades, trades], ignore_index=True).copy()

print(f'Total profit: ${all_trades.Profit_amount.sum():0.2f}')

[0/502][MMM] Winning trades:11/15 for total profit of $18.38
[1/502][AOS] Winning trades:9/22 for total profit of $-29.80
[2/502][ABT] Winning trades:8/26 for total profit of $-19.95
[3/502][ABBV] Winning trades:11/20 for total profit of $-13.26
[4/502][ACN] Winning trades:15/24 for total profit of $-33.56
[5/502][ATVI] Winning trades:9/31 for total profit of $-15.40
[6/502][ADM] Winning trades:6/28 for total profit of $-38.73
[7/502][ADBE] Winning trades:15/17 for total profit of $115.42
[8/502][ADP] Winning trades:15/27 for total profit of $-66.19
[9/502][AAP] Winning trades:10/14 for total profit of $23.17
[10/502][AES] Winning trades:9/23 for total profit of $-14.74
[11/502][AFL] Winning trades:11/19 for total profit of $-9.48
[12/502][A] Winning trades:12/19 for total profit of $-9.74
[13/502][APD] Winning trades:10/11 for total profit of $32.83
[14/502][AKAM] Winning trades:14/23 for total profit of $18.83
[15/502][ALK] Winning trades:10/18 for total profit of $-9.94
[16/502][ALB

In [12]:
# Display statsitics on Winning vs Losing trades
# Winning trades : 5459
# Losing trades : 4194
# Total trades : 9653
win_trades = all_trades.loc[all_trades.Profit].Symbol.count()
win_amount = all_trades.loc[all_trades.Profit].Profit_amount.sum()
loss_trades = all_trades.loc[all_trades.Profit == False].Symbol.count()
loss_amount = all_trades.loc[all_trades.Profit == False].Profit_amount.sum()

print(f'Winners: {win_trades} for ${win_amount:0.2f} amount')
print(f'Loosers: {loss_trades} for ${loss_amount:0.2f} amount')
print(f'Profit: ${(win_amount + loss_amount):0.2f}')

Winners: 5473 for $14724.65 amount
Loosers: 4201 for $-21334.31 amount
Profit: $-6609.66
